In [80]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.widgets import Slider
%matplotlib

Using matplotlib backend: MacOSX


In [18]:
def choose(n,k):
    assert type(n) == int and n >= 0, "Not a valid choice of n"
    assert (0 <= k <= n), "Not a valid choice of k"
    half = n // 2
    if k > half:
        k = n-k
    if k == 0:
        return 1
    elif k == 1:
        return n
    else:
        return choose(n-1,k-1) + choose(n-1,k)

In [19]:
print(choose(0,0))

1


In [76]:
def land(n):
    start = int(np.ceil(n/2))
    prob = 0
    for i in range(start,n+1):
        prob += choose(i,n-i)/(2**i)
    return prob

In [51]:
n = 9
mod = 2

class Style():
    Red = "\033[31m"
    Green = "\033[32m"
    Yellow = "\033[33m"
    Blue = "\033[34m"
    Magenta = "\033[35m"
    Reset = "\033[0m"

def col(u):
    n = 5
    modulo = u % n
    if modulo==0:
        return Style.Red
    elif modulo==1:
        return Style.Green
    elif modulo==2:
        return Style.Yellow
    elif modulo==3:
        return Style.Blue
    else:
        return Style.Magenta

for i in range(n+1):
    for j in range(0, n-i+1):
        print(' ', end='') 
    for j in range(i+1):
        u = choose(i,j)
        color = col(i+j)
        print('  ', f"{color}{u}{Style.Reset}", sep='', end='') 
    print(' ')

            1 
           1  1 
          1  2  1 
         1  3  3  1 
        1  4  6  4  1 
       1  5  10  10  5  1 
      1  6  15  20  15  6  1 
     1  7  21  35  35  21  7  1 
    1  8  28  56  70  56  28  8  1 
   1  9  36  84  126  126  84  36  9  1 


### Alternative formula for working out Y(n) ###

In [64]:
def alt_form(n):
    if n == 0:
        return [1], [1]
    elif n ==1:
        return [1], [1]
    else:
        p_1, o = alt_form(n-1)
        p_2 = alt_form(n-2)[0]
        
        p_2 = [0] + p_2
        if n%2 == 0:
            p_1 = p_1 + [0]
            o = o + [int(2**(n/2))]
        p = np.array(p_1) + np.array(p_2)
        return p.tolist(), o

def Y_alt_form(n):
    p,o = alt_form(n)
    #print(f'coefficients = {p}')
    #print(f'powers = {o}')
    pow = 1 / (2**n)
    y = 0
    for i in range(len(p)):
        y += pow * o[i] * p[i]
    return y
        

In [65]:
for j in range(1,7):
    print(Y_alt_form(j))

0.5
0.75
0.625
0.6875
0.65625
0.671875


In [33]:
for j in range(1,7):
    auto = land(j)
    #print(land(j))
    check = auto * 2**j
    print(auto, check)

0.5 1.0
0.75 3.0
0.625 5.0
0.6875 11.0
0.65625 21.0
0.671875 43.0


In [38]:
print(land(24))

0.6666666865348816


In [67]:
print(Y_alt_form(24))

0.6666666865348816


In [69]:
n0 = 24
Y_n = 0
r = -1/2
for i in range(n0+1):
    Y_n += r**i
print(Y_n)

0.6666666865348816


In [108]:
print(land(30))

0.6666666669771075


In [41]:
print(land(40))

0.6666666666669698


In [77]:
n0 = np.arange(0,25)
tol = 1e-15
truth = True
for n in n0:
    Y_n = 0
    r = -1/2
    for i in range(n+1):
        Y_n += r**i
    if np.abs(Y_n - land(n)) > tol:
        truth = False
        break
if truth:
    statement = 'matches'
else:
    statement = 'does not match'
print(f"The geometric formula I have presented {statement} my original binomial function")

The geometric formula I have presented matches my original binomial function


## Extending problem ##

In [114]:
def land_k(N,n_k_diff):
    prob_array = []
    for n in N:
        prob = 0
        k = n + n_k_diff
        start = int(np.ceil((k-1)/2))
        for i in range(start,n+1):
            prob += choose(i,k-1-i)/(2**i)
        prob_array = prob_array + [prob]
    return prob_array

In [124]:
for i in range(0,0+1):
    k = i + 2
    print(int(np.ceil((k-1)/2)))
    print(i+1)

1
1


In [118]:
N = np.arange(0, 26)
n_k_diff = 2

print(land_k(N, n_k_diff))

[0, 0.5, 0.5, 0.625, 0.625, 0.65625, 0.65625, 0.6640625, 0.6640625, 0.666015625, 0.666015625, 0.66650390625, 0.66650390625, 0.6666259765625, 0.6666259765625, 0.666656494140625, 0.666656494140625, 0.6666641235351562, 0.6666641235351562, 0.6666660308837891, 0.6666660308837891, 0.6666665077209473, 0.6666665077209473, 0.6666666269302368, 0.6666666269302368, 0.6666666567325592]


In [117]:
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.15)
plt.grid('on')
plt.ylim((0,0.8))
plt.xlabel('n')
plt.ylabel('$Y_{n+k}(n)$')
P, = plt.plot(N, land_k(N, n_k_diff))

plt.plot(N, 2/3*np.ones(len(N)), 'r--')

axk = plt.axes([0.23, 0.05, 0.55, 0.02])
k_slide = Slider(axk, '$k$', 1, 10,
          valstep = 1, valinit = 1)
def update(val):
    n_k_diff = k_slide.val
    P.set_ydata(land_k(N,n_k_diff))
    fig.canvas.draw_idle()
k_slide.on_changed(update)
plt.show()